# Distance, driving distance and duration between two places
Various implementation in python for the Earth surface distnace, travelling distance on roads and duration of such a journey. 

We will get help from the public dataset containing locations of all the world capitals from Kaggle - https://www.kaggle.com/nikitagrec/world-capitals-gps (public).

In [1]:
import pandas as pd
from geopy import distance
import requests # to call the openmap/google apis
import json
import datetime
import math
import itertools

# you may run into this problem https://stackoverflow.com/questions/61867945/python-import-error-cannot-import-name-six-from-sklearn-externals
# I have manually updated site-packages/mlrose/neural.py chaning 
# original:
# from sklearn.external import six
# to:
# import six
from mlrose import TravellingSales, TSPOpt, genetic_alg # for travelling salesman problem

# that was fixed in mlrose_hiive, though the outputs are different
# import mlrose_hiive
import numpy as np

In [ ]:
# load the dataframe with capitals
df = pd.read_csv("concap.csv")

# rename so that the column names are shorter and comply with PEP-8
df.rename(columns={"CountryName": "Country", "CapitalName": "capital", "CapitalLatitude": "lat", "CapitalLongitude": "lon", "CountryCode": "code", "ContinentName": "continent"}, inplace=True)
df.head(3)

Naming convetion of the variabled is described in PEP-8: https://www.python.org/dev/peps/pep-0008/#function-and-variable-names

There's discusion if it should be applied to the pandas columns as well, but I would suggest to do it - https://stackoverflow.com/questions/58584570/pep8-guidance-for-column-names-in-pandas-dataframe

In [ ]:
# to start with let's filter only 2 capitals. Rome and Paris.
ropa = df[df["capital"].isin(["Rome","Paris"])].reset_index()
cities = ropa.copy()
cities

## Calculating the distnace
The first obvious method is to use the shortest distnace on the surface of Earth. You can use various approximations:

* Great-circle distnace on the surface of sphere - https://en.wikipedia.org/wiki/Great-circle_distance
* Distances from geodesics since Earth is approximated as oblate ellipsoid https://en.wikipedia.org/wiki/Geodesics_on_an_ellipsoid
* Haversine formula - https://en.wikipedia.org/wiki/Haversine_formula, https://towardsdatascience.com/calculating-distance-between-two-geolocations-in-python-26ad3afe287b

You don't have to invent or even reproduce this math. The geopy.distance module already implemented all of these distnance calculation, it returns the values in kilometers (km), miles (mi), nautical miles (nm) or feet (ft).  All these methods are part of `distance` class we have already imported from geopy.
* `distance((latitude_point_1, longitude_point_1), (lat_2, lon_2))` - using geodesic on `WGS-84` ellipsoid
* `geodesic((latitude_point_1, longitude_point_1), (lat_2, lon_2))`
* `great_circle((latitude_point_1, longitude_point_1), (lat_2, lon_2))`

More info about geopy.distance https://geopy.readthedocs.io/en/stable/#module-geopy.distance

In [ ]:
d = distance.distance((cities.loc[0, "lat"], cities.loc[0, "lon"]), (cities.loc[1, "lat"], cities.loc[1, "lon"]))
d, d.km, d.miles

In [ ]:
getattr(d, "km")

In [ ]:
results = []
for f in [distance.distance, distance.great_circle, distance.geodesic]:
    for mes in ["kilometers","km","miles","mi","nautical","nm","feet","ft"]:
        d = f((cities.loc[0, "lat"], cities.loc[0, "lon"]), (cities.loc[1, "lat"], cities.loc[1, "lon"]))
        results.append({"method": f.__name__, "measurement": mes, "value": getattr(d, mes)})

# show as dataframe
results_df = pd.DataFrame(results)
results_df.pivot_table(index="method", columns="measurement", values="value")

`distance.distance` nativelly calls `distance.geodesic` that's why these two calues collapse into one row in the results. 

In [ ]:
# the distnace for various ellipsiods
for ellipsoid in distance.ELLIPSOIDS:
    for mes in ["kilometers","km","miles","mi","nautical","nm","feet","ft"]:
        d = distance.geodesic((cities.loc[0, "lat"], cities.loc[0, "lon"]), (cities.loc[1, "lat"], cities.loc[1, "lon"]), ellipsoid=ellipsoid)
        results.append({"method": f"geodesic: {ellipsoid}", "measurement": mes, "value": getattr(d, mes)})

# show as dataframe
results_df = pd.DataFrame(results)
results_df.pivot_table(index="method", columns="measurement", values="value")

# Driving distance
The cities can be quite close on the surface, though natural obstacles like sea or mountain can cause that the driving distance is much longer. 

In [ ]:
hest = df[df["capital"].isin(["Helsinki","Stockholm"])].reset_index()
cities = hest.copy()
d = distance.distance((cities.loc[0, "lat"], cities.loc[0, "lon"]), (cities.loc[1, "lat"], cities.loc[1, "lon"]))
d.km

Even though the distance between Helsinky, the capita of Finland and Stockholm in Sweden less than 400km, if you decide to drive it's more than 1750km and 20 hours. Even if you take ferries you will drive almost 500km. Paris is located only 1107km from Rome, but roads connecting these cities have at least 1420km. 


That's why for many application you want to know the real travel distnace, which no mathematical function can return. You need to call some map service API - e.g. google routes or osrm route service (http://project-osrm.org/docs/v5.5.1/api/#route-service). The documentation says that in order to get the driving distance we need to call this API endoint - `/route/v1/{profile}/{coordinates}?alternatives={true|false}&steps={true|false}&geometries={polyline|polyline6|geojson}&overview={full|simplified|false}&annotations={true|false}` having parameters: 

* `profile` - car, bike, foot 
* `coordinates` - lat,lon of the first point; lon, lat of the second point, e.g. 2.333333,48.866667;12.483333,41.900000
* `alternative` - whether to return only the first option or more alternatives
* `steps` - whether to return route steps - e.g. at the crossroad turn left
* `geometries` - how is the route returned, either `polyline` (def`ault), `polyline6` ,  `geojson`
* `overview` - how  to return the route - `simplified` (default), `full`,  `false`
* `annotations` - if aditional metadata are provided for each point on the route

For our purpose, we can run the simplest request, having everything set to false or default. We will simply call: `http://router.project-osrm.org/route/v1/driving/cities.loc[0, "lon"],cities.loc[0, "lat"];cities.loc[1, "lon"],cities.loc[1, "lat"]?overview=false`

To get the API's response, we will use the python's requests method. 

In [ ]:
cities = ropa.copy()
r = requests.get(f"""http://router.project-osrm.org/route/v1/car/{cities.loc[0, "lon"]},{cities.loc[0, "lat"]};{cities.loc[1, "lon"]},{cities.loc[1, "lat"]}?overview=false""")

It returns whatever is provided by the API in the `content` parameter.

In [ ]:
r.content

We can see that it returns a json object, which looks nices, when passed to the python's json library. 

In [ ]:
json.loads(r.content)

We're mainly interested about the driving `distance` and/or driving `duration`. These parameters are included into the `route` subelement which contains a list of routes. Because we haven't asked for any alternative, this list has only one item. 

In [ ]:
route_1 = json.loads(r.content)["routes"][0]
route_1["distance"], route_1["duration"]

If the resulted numbers look strange, beware that the distnace is in meters and duration in seconds. You can easily check that the values are correct online - https://www.openstreetmap.org/directions?engine=fossgis_osrm_car&route=48.867%2C2.333%3B41.900%2C12.484 (for driving distnace by car from Paris to Rome). 

Maybe you prefer more human readable format. That can be achieved by passing the received duration as a parameter to `timedelta` function of `datetime` and return the string representation. In case it's more than 1 day (I've added 100000 seconds the Paris-Rome distnace for that purpose), than it's displayed.  

In [ ]:
x = datetime.timedelta(seconds=route_1["duration"]+100000)

In [ ]:
str(x)

You can also use pandas, if you specify the type of the `duration` column to be `timedelta64[s]`.

In [ ]:
dftime = pd.DataFrame({"duration":route_1["duration"]+100000}, index=[0])
dftime["duration"] = dftime["duration"].astype("timedelta64[s]")
dftime

You should use the OSRM project responsibly, because it's there to server everyone. Read the rules of the service https://github.com/Project-OSRM/osrm-backend/wiki/Api-usage-policy. 

If you are building a business application which needs hundreds or thousands of route requests, opt for a commercial product like google direction service. https://developers.google.com/maps/documentation/directions/overview. Again you use the requests library, specify correct url and get the result. Beware that you need to make an agreement with Google and these requests are paid. You have to prove yourself with valid API key and you are billed according to the service policies. Usually these services offer some free amount of searches, which can be used for your proof of concept. E.g. google offer \\$200/month worth of credit, while 1K requests costs \\$10. 

In order to avoid surprises, always restrict your API keys to the purpose you need it for and set up the billing ceiling. 

In [ ]:
origin_coor = ",".join([str(cities.loc[0,"lat"]), str(cities.loc[0,"lon"])])
destination_coor = ",".join([str(cities.loc[1,"lat"]), str(cities.loc[1,"lon"])])
API_KEY = "api_key"
url = f"https://maps.googleapis.com/maps/api/directions/json?origin={origin_coor}&destination={destination_coor}&mode=driving&key={API_KEY}"

# alternatively you can specify the start point (origin) and the destination using the places' names
url_alt = f"https://maps.googleapis.com/maps/api/directions/json?origin=Paris&destination=Rome&mode=driving&key={API_KEY}"
print(url)

In [ ]:
r = requests.get(url_alt)

In this case you find the results in the the appropriate keys of the response. 

In [ ]:
results = json.loads(r.content)
legs = results["routes"][0]["legs"]
legs[0]["duration"], legs[0]["distance"]

# Distances between multiple cities and optimal route
The route doesn't have to be limited to two cities only. Maybe you need to visit several addreses/cities and you're trying to optimize the process. Let's try on an example of exploring all the capitals of the Central Europe. 

First we will list these capitals. Looking to Wikipedia (https://en.wikipedia.org/wiki/Central_Europe) you will find that Central Europe include 9 countries - Austria, Czech Republic, Germany, Hungary, Liechtenstein, Poland, Slovenia, Slovakia, Switzerland. I've listed their ISO2 codes below. 

In [ ]:
ce_countries = ["AT","CZ","DE","HU","LI","PL","SK","SI","CH"]

In [ ]:
ce_cities = df[df["code"].isin(ce_countries)].reset_index(drop=True)
ce_cities

Let's wrap the OSRM distnace service into a function.

In [ ]:
def get_distance(point1: dict, point2: dict) -> tuple:
    """Gets distance between two points en route using http://project-osrm.org/docs/v5.10.0/api/#nearest-service"""
    
    url = f"""http://router.project-osrm.org/route/v1/driving/{point1["lon"]},{point1["lat"]};{point2["lon"]},{point2["lat"]}?overview=false&alternatives=false"""
    r = requests.get(url)
    
    # get the distance from the returned values
    route = json.loads(r.content)["routes"][0]
    return (route["distance"], route["duration"])

In [ ]:
# let's try on the first two cities
# confirm that it's correct on https://www.openstreetmap.org/directions?engine=fossgis_osrm_car&route=48.208%2C16.372%3B50.087%2C14.421
get_distance({"lat": 48.200000,"lon": 16.366667}, {"lat": 50.083333,"lon": 14.466667})

Now we can run the distnace calculation for all the combinations of the cities. There's 
\begin{equation*} 
{9 \choose 2} 
\end{equation*}
of combinations.

In [ ]:
# https://stackoverflow.com/questions/464864/how-to-get-all-possible-combinations-of-a-list-s-elements
# https://stackoverflow.com/questions/3025162/statistics-combinations-in-python
len([c for c in itertools.combinations(list(cities["capital"]),2)])

In [ ]:
# from python 3.8 you can use math.comb to get just the number. 
# math.comb(9,2)

Let's get the distance and duration from OSRM API for all our combinations.

In [ ]:
dist_array = []
for i , r in ce_cities.iterrows():
    point1 = {"lat": r["lat"], "lon": r["lon"]}
    for j, o in ce_cities[ce_cities.index != i].iterrows():
        point2 = {"lat": o["lat"], "lon": o["lon"]}
        dist, duration = get_distance(point1, point2)
        #dist = geodesic((i_lat, i_lon), (o["CapitalLatitude"], o["CapitalLongitude"])).km
        dist_array.append((i, j, duration, dist))

In [ ]:
distances_df = pd.DataFrame(dist_array,columns=["origin","destination","duration(s)","distnace(m)"])
distances_df

In [ ]:
distances_df = distances_df.merge(ce_cities[["capital"]], left_on = "origin", right_index=True).rename(columns={"capital":"origin_name"})
distances_df = distances_df.merge(ce_cities[["capital"]], left_on = "destination", right_index=True).rename(columns={"capital":"destination_name"})
distances_df

# Travelling Salesperson Problem
To find the optimal route between a list of points based on a parameter (e.g. duration or distance between the places) we can use the Travelling Salesperson Problem (https://en.wikipedia.org/wiki/Travelling_salesman_problem). Travelling Sales person wants to visit all the points with the minimal effort. 

Python implementation is described in the following article. https://towardsdatascience.com/solving-travelling-salesperson-problems-with-python-5de7e883d847. You can also read the documentation of the mlrose package https://mlrose.readthedocs.io/en/stable/source/tutorial2.html.


In [ ]:
# we plan to visit 9 cities
length = ce_cities.shape[0]

we plan to use the list of distnaces (durations in our case), that's why we initialize with `distances = dist_list` param. E.g. distance between the `place 0` and `place 1` is `13949.0` second --> `(0,1,13949.0)`

In [ ]:
# turn the first three columns of the dataframe into the list of tuples
dist_list = list(distances_df[["origin","destination","duration(s)"]].sort_values(by=["origin","destination"]).to_records(index=False))
dist_list

In [ ]:
# we plan to use the list of distnaces (durations in our case), that's why we initialize with `distances = dist_list` param.
fitness_dists = mlrose.TravellingSales(distances = dist_array)

In [ ]:
problem_fit = mlrose.TSPOpt(length = length, fitness_fn = fitness_dists,
                            maximize=False)

In [ ]:
mlrose.genetic_alg(problem_fit, random_state = 2)

In [ ]:
# Solve problem using the genetic algorithm - suboptimal solution
best_state, best_fitness = mlrose.genetic_alg(problem_fit, random_state = 2)

In [ ]:
print(f"The best state found is: {best_state}, taking {best_fitness} ({str(datetime.timedelta(seconds=best_fitness))})")

In [ ]:
# better but more resource intensive solutions
best_state, best_fitness = mlrose.genetic_alg(problem_fit, mutation_prob = 0.2,  max_attempts = 500, random_state = 2)

In [ ]:
print(f"The best state found is: {best_state}, taking {best_fitness} ({str(datetime.timedelta(seconds=best_fitness))})")

The `best_state` contains the order of the places to visit. Let's map it to our list of cities to see, the order in which we can to visit them.

In [ ]:
orders = {city: order for order, city in enumerate(best_state)}
orders

In [ ]:
ce_cities["order"] = ce_cities.index.map(orders)
ce_cities = ce_cities.sort_values(by="order")
ce_cities

Let's confirm that the distance is really the `best_fitness`. Based on the order we will add the `next_city` column and specially handle the last city which is followed by the city with order == 0 (or the minimum order).

In [ ]:
ce_cities["next_city"] = ce_cities["capital"].shift(-1)

# the last connection is between the last city and the first one
ce_cities.loc[ce_cities["order"] == max(ce_cities["order"]), "next_city"] = ce_cities.loc[ce_cities["order"] == min(ce_cities["order"]), "capital"].values[0]
ce_cities

Then we join the distances in seconds from `distance_df`

In [ ]:
ordered_ce_cities = ce_cities.merge(distances_df[["origin_name","destination_name","duration(s)"]], left_on=["capital","next_city"], right_on=["origin_name","destination_name"], how="left")

In [ ]:
ordered_ce_cities

Let's check that the total of this distance is really, what Travelling Salesman Problem identified as the `best_fitness`

In [ ]:
ordered_ce_cities["duration(s)"].sum()

Since the route is a cyclical you can start at any of the capitals. To get the minimal travel time, you should end in the city which is followed by the longest duration. 

In [ ]:
ordered_ce_cities["duration(s)"].max()

In [ ]:
ordered_ce_cities.style.highlight_max(color = 'lightgreen', axis = 0, subset="duration(s)")

As you can see, in our case, if you can start in Bern and end in Ljublana, the distance would be the smallest.

In [ ]:
duration_s = ordered_ce_cities["duration(s)"].sum() - ordered_ce_cities["duration(s)"].max()
duration_s, str(datetime.timedelta(seconds=duration_s))

# Confirm that this is really the shorted possible route

In [ ]:
l = [0,1,2,3,4,5,6,7,8]

In [ ]:
# There's possible (n-1)!/2 combinations; 1/2 because path 1-2-3 is the same as 3-2-1
math.factorial(length)/2

Let's generate all the possible paths

In [ ]:
perm = [l for l in itertools.permutations(l, 9)]
len(perm)

In [ ]:
perm[:5]

In [ ]:
distances = {(int(d[0]), int(d[1])):d[2] for d in distances_df[["origin","destination","duration(s)"]].values.tolist()}
distances[(0,1)]

In [ ]:
# pick the first path
path = perm[0]

# add th first element to conclude the circular path
path = path + (path[0],)

# iterate through the path and sum all the distnaces
total_path_distance = 0
for i in range(len(path)-1):
    edge = (path[i], path[i+1])
    total_path_distance += distances[edge]
    print(edge, distances[edge])
print(total_path_distance)

# list comprehension
sum([distances[(path[i],path[i+1])] for i in range(len(path)-1)])

In [ ]:
mn = np.inf
min_paths = []

for p in perm:
    
    # add th first element to conclude the circular path
    p = p + (p[0],)
    
    total_path_distance = sum([distances[(p[i],p[i+1])] for i in range(len(path)-1)])
    
    if total_path_distance < mn:
        mn = total_path_distance
        min_paths = [p]
    elif total_path_distance == mn:
        min_paths.append(p)
    else:
        pass
    
print(mn, min_paths)

In [ ]:
def path_to_df(path, cities, distances_df):
    orders = {city: order for order, city in enumerate(path[0])}
    
    cities["order"] = cities.index.map(orders)
    cities = cities.sort_values(by="order")
    
    cities["next_city"] = cities["capital"].shift(-1)

    # the last connection is between the last city and the first one
    cities.loc[cities["order"] == max(cities["order"]), "next_city"] = cities.loc[cities["order"] == min(cities["order"]), "capital"].values[0]
    
    ordered_ce_cities = cities.merge(distances_df[["origin_name","destination_name","duration(s)"]], left_on=["capital","next_city"], right_on=["origin_name","destination_name"], how="left")
    
    return ordered_ce_cities["duration(s)"].sum(), ordered_ce_cities

In [ ]:
total_duration, path_df = path_to_df(min_paths, ce_cities, distances_df)
path_df.style.highlight_max(color = 'lightgreen', axis = 0, subset="duration(s)")

In [ ]:
total_duration

Let's shift the order so that the path start in Ljubljana

In [ ]:
filter_cities_before_bern = path_df["order"] < path_df.loc[path_df["capital"]=="Ljubljana","order"].values[0]
path_df.loc[filter_cities_before_bern, "order"] += path_df["order"].max()
path_df.sort_values(by="order")

Let's now reverse the order, so that we won't go from Ljublana to Vaduz, but vice versa

In [ ]:
path_df_reversed = path_df.sort_values(by="order").reset_index(drop=True)
path_df_reversed = path_df_reversed.reindex(index=path_df_reversed.index[::-1])
path_df_reversed

This path is exactly the same our Travelling Salesman algorithm found. You can see in the `min_paths` variable, that all combinations of these roads are the shortest (considering that we return to the place where we have started).